<a href="https://colab.research.google.com/github/Shadhujan/Baseline_FER/blob/main/FER_mistralAI_11052025.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
!pip install mtcnn tensorflow==2.12


In [3]:
import os
import numpy as np
import tensorflow as tf
from mtcnn import MTCNN
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras import layers, models, callbacks
from tensorflow.keras.applications import VGG16
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Dense, GlobalAveragePooling2D, Dropout

In [4]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [7]:
import zipfile
from pathlib import Path

# 1. Paths to your zip files
zip_paths = [
    "/content/drive/MyDrive/datasets/FER-2013_v1.zip",
    "/content/drive/MyDrive/datasets/FER-2013_v2.zip",
    "/content/drive/MyDrive/datasets/archive.zip",
    "/content/drive/MyDrive/datasets/fer2013.csv.zip"
]

# 2. Create the dataset directory
dataset_dir = Path("/content/dataset")
dataset_dir.mkdir(parents=True, exist_ok=True)

# 3. Extract each zip
for zp in zip_paths:
    with zipfile.ZipFile(zp, 'r') as archive:
        archive.extractall(dataset_dir)
        print(f"Extracted {zp} ➔ {dataset_dir}")

Extracted /content/drive/MyDrive/datasets/FER-2013_v1.zip ➔ /content/dataset
Extracted /content/drive/MyDrive/datasets/FER-2013_v2.zip ➔ /content/dataset
Extracted /content/drive/MyDrive/datasets/archive.zip ➔ /content/dataset
Extracted /content/drive/MyDrive/datasets/fer2013.csv.zip ➔ /content/dataset


In [8]:
# 2. Data Generators
train_datagen = ImageDataGenerator(
    rescale=1./255,
    rotation_range=15,
    width_shift_range=0.1,
    height_shift_range=0.1,
    zoom_range=0.1,
    horizontal_flip=True
)

test_datagen = ImageDataGenerator(rescale=1./255)

train_gen = train_datagen.flow_from_directory(
    '/content/dataset/train',
    target_size=(48, 48),
    batch_size=32,
    class_mode='categorical'
)

test_gen = test_datagen.flow_from_directory(
    '/content/dataset/test',
    target_size=(48, 48),
    batch_size=32,
    class_mode='categorical'
)

Found 28709 images belonging to 7 classes.
Found 7178 images belonging to 7 classes.


In [9]:
# Face Detection & Cropping
detector = MTCNN()
def extract_face(img):
    res = detector.detect_faces(img)
    if res:  # Check if a face is detected
        x, y, w, h = res[0]['box']
        return img[y:y+h, x:x+w]
    return img  # Return the original image if no face is detected

def preprocess(img):
    face = extract_face(img)
    face = tf.image.resize(face, [48, 48]) / 255.0
    return face


In [10]:
# Load pre-trained VGG16 model
base_model = VGG16(weights='imagenet', include_top=False, input_shape=(48, 48, 3))

# Freeze the layers of the base model
for layer in base_model.layers:
    layer.trainable = False

# Add custom layers on top
x = base_model.output
x = GlobalAveragePooling2D()(x)
x = Dense(1024, activation='relu')(x)
x = Dropout(0.5)(x)
predictions = Dense(train_gen.num_classes, activation='softmax')(x)


58889256/58889256 [==============================] - 0s 0us/step


In [ ]:
# Create the final model
model = Model(inputs=base_model.input, outputs=predictions)

# Compile the model
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

# Early Stopping & Fit
early_stop = callbacks.EarlyStopping(
    monitor='val_loss',
    patience=5,
    restore_best_weights=True
)

history = model.fit(
    train_gen,
    validation_data=test_gen,
    epochs=50,
    callbacks=[early_stop]
)

Epoch 1/50
267/898 [=======>......................] - ETA: 5:15 - loss: 1.7655 - accuracy: 0.2928

In [ ]:
# Plot Metrics
import matplotlib.pyplot as plt

plt.plot(history.history['accuracy'], label='train_acc')
plt.plot(history.history['val_accuracy'], label='test_acc')
plt.legend()
plt.show()

In [ ]:
# Confusion Matrix
from sklearn.metrics import confusion_matrix, classification_report

# Gather all test data and true labels
X_test, y_test = [], []
for X_batch, y_batch in test_gen:
    X_test.append(X_batch)
    y_test.append(y_batch)
    if len(X_test)*test_gen.batch_size >= test_gen.samples:
        break

X_test = np.vstack(X_test)
y_true = np.argmax(np.vstack(y_test), axis=1)
y_pred = np.argmax(model.predict(X_test), axis=1)

cm = confusion_matrix(y_true, y_pred)
print(cm)
print(classification_report(y_true, y_pred, target_names=list(test_gen.class_indices.keys())))